# All Libraries Imported

In [11]:
#Importing all necessary libraries
#!pip install seaborn
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
import sklearn
import nltk.collocations 
from nltk import FreqDist, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string, re
import urllib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
#nltk.download('punkt')
#nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Read The CSV File

In [12]:
song_list = pd.read_csv(r'')#insert csv file location


# Remove rows with N/A or Other

In [13]:
song_list.genre.value_counts()


Rock          100052
Pop            34137
Hip-Hop        22654
Metal          21210
Country        14158
Jazz            7310
Electronic      6942
Other           3786
R&B             3336
Indie           2935
Folk            1689
Name: genre, dtype: int64

In [14]:
song_list.drop(song_list[song_list.genre == "Other"].index, inplace = True)
song_list.drop(song_list[song_list.genre == "Not Available"].index, inplace = True)
song_list.drop(song_list[song_list.genre == "Metal"].index, inplace = True)
song_list.drop(song_list[song_list.genre == "Indie"].index, inplace = True)
song_list.drop(song_list[song_list.genre == "Folk"].index, inplace = True)
song_list.drop(song_list[song_list.artist == "arcade-fire"].index, inplace = True)
song_list.drop(song_list[song_list.artist == "funkadelic"].index, inplace = True)
song_list.shape

(209632, 7)

# Drop songs with NaN value for song 

In [15]:
song_list.dropna(inplace = True)
song_list.song.isna().sum()
song_list.reset_index(inplace = True)

In [16]:
song_list.tail()

,level_0,Unnamed: 0,index,song,year,artist,genre,lyrics
209626,218204,362232,362232,who-am-i-drinking-tonight,2012,edens-edge,Country,I gotta say Boy after only just a couple of da...
209627,218205,362233,362233,liar,2012,edens-edge,Country,I helped you find her diamond ring You made me...
209628,218206,362234,362234,last-supper,2012,edens-edge,Country,Look at the couple in the corner booth Looks a...
209629,218207,362235,362235,christ-alone-live-in-studio,2012,edens-edge,Country,When I fly off this mortal earth And I'm measu...
209630,218208,362236,362236,amen,2012,edens-edge,Country,I heard from a friend of a friend of a friend ...


# Clean titles of songs

In [17]:
def clean_song_names(titles_list):
    no_dot = list(map(lambda item: item.replace(".", ""), titles_list))
    no_dash = list(map(lambda item: item.replace("-", " "), no_dot))
    return list(map(lambda item: string.capwords(item), no_dash))

titles_list = song_list.song
final_titles = clean_song_names(titles_list)
len(final_titles)

209631

# Drop old columns

In [18]:
new_df = song_list
# new_df.head()
new_titles = final_titles


new_lyr = pd.DataFrame(new_titles)
new_lyr.tail()

final_df = new_df.join(new_lyr)

final_df.drop(columns = ['song', 'level_0', "index"], axis = 1, inplace = True)

In [19]:
final_df.shape

(209631, 6)

In [20]:
final_df.rename(columns = {0: "song"}, inplace = True)
final_df.isna().sum()

Unnamed: 0    0
year          0
artist        0
genre         0
lyrics        0
song          0
dtype: int64

In [21]:
final_df.shape

(209631, 6)

In [31]:
rock_df = final_df[final_df.genre == "Rock"][-900:]
pop_df = final_df[final_df.genre == "Pop"][:900]
hip_df = final_df[final_df.genre == "Hip-Hop"][:900]
jazz_df = final_df[final_df.genre == "Jazz"][:900]
elec_df = final_df[final_df.genre == "Electronic"][:900]
country_df = final_df[final_df.genre == "Country"][:900]
rnb_df = final_df[final_df.genre == "R&B"][:900]

In [23]:
rock_df.head

<bound method NDFrame.head of         Unnamed: 0  year      artist genre  \
206753      357439  2009  brainstorm  Rock   
206754      357440  2009  brainstorm  Rock   
206755      357441  2007  brainstorm  Rock   
206756      357442  2007  brainstorm  Rock   
206757      357443  2007  brainstorm  Rock   
...            ...   ...         ...   ...   
209593      362164  2011    corroded  Rock   
209594      362165  2011    corroded  Rock   
209595      362166  2011    corroded  Rock   
209596      362167  2011    corroded  Rock   
209597      362168  2011    corroded  Rock   

                                                   lyrics                song  
206753  More or less Everyone has a mess In their hear...           And I Lie  
206754  Easy we follow easy we trust Some things we wa...      French Cartoon  
206755   every word that you say every lie i can hear ...  Doorway To Survive  
206756   the leading to where no one has nothing to sa...              Fading  
206757  you can c

In [24]:
final_df.drop(final_df[final_df.genre == "Rock"].index, inplace = True)
final_df.drop(final_df[final_df.genre == "Pop"].index, inplace = True)
final_df.drop(final_df[final_df.genre == "Hip-Hop"].index, inplace = True)
final_df.drop(final_df[final_df.genre == "Jazz"].index, inplace = True)
final_df.drop(final_df[final_df.genre == "Electronic"].index, inplace = True)
final_df.drop(final_df[final_df.genre == "Country"].index, inplace = True)

In [32]:
maybe_df = final_df.append([rock_df, pop_df, hip_df, country_df, jazz_df, elec_df])
maybe_df.genre.value_counts()

Metal         21210
Jazz           8209
Electronic     7842
R&B            3336
Name: genre, dtype: int64

In [33]:
maybe_df = final_df.append([rock_df, pop_df, hip_df, jazz_df, elec_df, country_df])
maybe_df.genre.value_counts()
maybe_df.drop(maybe_df[maybe_df.genre == "R&B"].index, inplace = True)
maybe_df = maybe_df.append([rnb_df])

In [34]:
maybe_df.genre.value_counts()

Metal         21210
Jazz           8209
Electronic     7842
R&B             900
Name: genre, dtype: int64

In [35]:
maybe_df.reset_index(inplace = True)
maybe_df.drop(columns = ["index"], axis = 1, inplace = True)
maybe_df.head()

,Unnamed: 0,year,artist,genre,lyrics,song
0,415,2011,demonaz,Metal,Gods Of The Mountains Sky Forest And Seas Land...,Northern Hymn
1,416,2011,demonaz,Metal,Rise gods of the fierous black burning skies R...,Under The Great Fires
2,417,2011,demonaz,Metal,A Dying Skyline Cold From Wind And Rain Autumn...,All Blackened Sky
3,418,2011,demonaz,Metal,Fiends of the gods to war we ride Over the bla...,Legends Of Fire And Ice
4,419,2011,demonaz,Metal,Cold Is The Winter On Its Ground We Ride To Th...,Ode To Battle


# Lemmatize

In [37]:
lyrics_list = list(maybe_df.lyrics)

pattern = "([a-zA-Z]+(?:'[a-z]+)?)"

lemmatizer = WordNetLemmatizer()

english = list(set(nltk.corpus.words.words()))

def clean_docs_lemma(lyrics_list):
    cleaned = []
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    for lyric in lyrics_list:
        clean_lyric = nltk.regexp_tokenize(lyric, pattern)
        lyric_lower = [i.lower() for i in clean_lyric]
        stop_words = stopwords.words('english')
        stopwords_list = stop_words + list(string.punctuation)
        stopwords_list += ["''", '""', '...', '``']
        lyrics_tokens_stopped = [w for w in lyric_lower if not w in stopwords_list]
        lyric_lemmas = [lemmatizer.lemmatize(word) for word in lyrics_tokens_stopped]
        c = " ".join(lyric_lemmas)
        cleaned.append(c)
    return cleaned

def clean_docs_stemma(lyrics_list):
    cleaned = []
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    for lyric in lyrics_list:
        clean_lyric = nltk.regexp_tokenize(lyric, pattern)
        lyric_lower = [i.lower() for i in clean_lyric]
        stop_words = stopwords.words('english')
        stopwords_list = stop_words + list(string.punctuation)
        stopwords_list += ["''", '""', '...', '``']
        lyrics_tokens_stopped = [w for w in lyric_lower if not w in stopwords_list]
        lyric_stemmas = [stemmer.stem(word) for word in lyrics_tokens_stopped]
        c = " ".join(lyric_stemmas)
        cleaned.append(c)
    return cleaned
